# Data

## EDA (Explorasi Data Analysis)  
> proses analisis awal yang dilakukan pada dataset untuk memahami karakteristik, pola, dan struktur data sebelum melakukan analisis lebih lanjut atau membangun model.

### Download dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import gdown
import string

In [ ]:
name = 'data_crawling_berita_antaranews.csv'
gdown.download(f'https://drive.google.com/uc?id=1iuY-raVaRjcwV63Ua8fIGa9dC01yvPvQ', name, quiet=False)

### Baca dataset

In [ ]:
data = pd.read_csv("data_crawling_berita_antaranews.csv")
data

#### Banyak dataset

In [ ]:
len(data)

#### Banyak dataset setiap kelas

In [ ]:
count_ekonomi = 0
count_olahraga= 0
count_politik = 0

for value in data['Label']:
  if value == 'ekonomi':
    count_ekonomi+=1
  elif value == 'olahraga':
    count_olahraga+=1
  else:
    count_politik += 1

print(
f'''
Banyak data ekonomi = {count_ekonomi} data
Banyak data olahraga= {count_olahraga} data
Banyak data politik = {count_politik} data

Total Data          = {len(data)} data
'''
)

## Preprocessing

### Missing Value  
> merupakan nilai pada sebuah data yang kosong/none sehingga harus dihapus untuk proses lebih lanjut

In [ ]:
data = data.dropna()
data

### Duplicate

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()
data


In [ ]:
data.reset_index(drop=True, inplace=True) #atur indeks data mulai dari 0 lagi

### **Punctuation process**

> **Punctuation process** merupakan proses normalisasi data yang bertujuan untuk menghilangkan tanda baca



In [ ]:
import re
data['clean_artikel'] = data['Artikel'].str.replace(r'[^\w\s,.?!]', '', regex=True).str.lower()

# Menghilangkan angka dari kolom 'new_abstrak'
data['clean_artikel'] = data['clean_artikel'].str.replace('\d+', '', regex=True)

In [ ]:
data

### **Stopword**

> Stopwords digunakan untuk menghilangkan kata umum yang sering muncul dalam teks seperti: di, dan, atau, dari, ke, saya.



In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

# Download kamus stop words
nltk.download('stopwords')

In [ ]:
# Inisialisasi kamus stop words dari NLTK
stop_words = set(stopwords.words('indonesian'))  # Inisialisasi kamus stop words

# Menghapus stop words dari kolom 'Abstrak'
for stop_word in stop_words:
  data['artikel_stopword'] = data['clean_artikel'].str.replace(rf'\b{stop_word}\b', '', regex=True) #rf untuk formating string

In [ ]:
data

### Tokenizing  

> **Tokenizing** adalah proses memecah teks atau dokumen menjadi potongan-potongan yang lebih kecil, yang disebut token. Disini menggunakan term kalimat

In [ ]:
data["tokenizing"] = data['artikel_stopword'].apply(sent_tokenize)

In [ ]:
data

In [ ]:
len(data['tokenizing'][0])

## VSM

### Term Freq

> Term Freq adalah konsep yang digunakan dalam pemrosesan teks dan analisis teks untuk mengukur sejauh mana suatu kata atau term muncul dalam sebuah dokumen atau koleksi dokumen. Term frequency menggambarkan seberapa sering sebuah kata muncul dalam teks relatif terhadap total kata dalam dokumen tersebut.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
hasil_df_tf = []
set_len_doc = data['tokenizing']

for i in range(len(set_len_doc)): #perulangan setiap artikel/dokumen
  termFreq_vectorizer = CountVectorizer()  #inisialisasi tfidf

  termFreq_matrix = termFreq_vectorizer.fit_transform(data['tokenizing'][i]) #menjadikan tfidf setiap dokumen
  terms = termFreq_vectorizer.get_feature_names_out() #ambil nama fitur

  tf_df = pd.DataFrame(termFreq_matrix.toarray(), columns=terms) #menjadikan dataframe
  hasil_df_tf.append(tf_df) #menyimpan dalam variabel df_tf

In [ ]:
# menampilkan hasil banyak kata pada kalimat

for i in range(len(hasil_df_tf[:10])):
  show_df = hasil_df_tf[i]
  print(f"========= Dokumen ke - {i} ==============")
  display(show_df)
  print("\n")

## Co-occurrence metrics  
> merupakan metrik kemunculan bersama merujuk pada ukuran-ukuran atau metode-metode yang digunakan untuk mengukur sejauh mana dua atau lebih elemen muncul bersama-sama dalam sebuah dataset. Ini dapat diterapkan dalam berbagai konteks, tergantung pada jenis data dan tujuan analisisnya. Berikut adalah beberapa contoh metrik kemunculan bersama yang umum digunakan:

In [ ]:
hasil_co_occurrence_df = [] #untuk menyimpan dataframe co-occurance
hasil_co_occurrence_matrix = [] #menyimpan hasil matrixnya

for i in range(len(hasil_df_tf)):
  # Membuat Co-occurrence Matrix
  co_occurrence_matrix = np.dot(hasil_df_tf[i].T,hasil_df_tf[i])

  # Mengganti diagonal dengan nol (karena kita tidak ingin memperhitungkan kata dengan dirinya sendiri)
  np.fill_diagonal(co_occurrence_matrix, 0)

  # mengambil columns
  kolom = hasil_df_tf[i].columns

  # # Membuat DataFrame Co-occurrence
  co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=kolom, columns=kolom)

  hasil_co_occurrence_df.append(co_occurrence_df) #simpan dataframe
  hasil_co_occurrence_matrix.append(co_occurrence_matrix) #simpan hasil matrix


In [ ]:
#menampilkan hasil co-occurence

for i in range(len(hasil_co_occurrence_df[:10])):
  print(f"============== Dokumen ke - {i} ==============")
  display(hasil_co_occurrence_df[i])
  print('\n')

## Tambahkan ke graf dan beri treshold

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
graf_results = []
treshold = 0

for i in range(len(hasil_co_occurrence_df)):

  graf = nx.Graph()  # Instantiate as a Graph object

  co_occurrence = hasil_co_occurrence_df[i] #mengambil hasil coocurence per indeks yang disimpan array
  koloms = co_occurrence.columns #mengambil nama kolom saja dari coocurence array indeks
  matrix_coocurrence = hasil_co_occurrence_matrix[i] #mengambil matrix coocurencenya

  for i_koloms in range(len(koloms)): #perulangan setiap dari panjang kolom dari coocurence
      for j_koloms in range(i_koloms + 1, len(koloms)): #perulangan dari setiap panjang kolom + 1, maksudna kolom indeks ke dua
        bobot = matrix_coocurrence[i_koloms,j_koloms] #mencari bobot dengan setiap kolom

        if bobot > treshold: #cek apakah bobot lebih dari treshold
          graf.add_edge(koloms[i_koloms], koloms[j_koloms], weight=matrix_coocurrence[i_koloms, j_koloms])

  graf_results.append(graf)


In [ ]:
# hasil_co_occurrence_df[:1]

In [ ]:
# # Menetapkan treshold
# treshold = 0

# # Inisialisasi list untuk menyimpan hasil graf
# graf_results = []

# # Iterasi melalui setiap DataFrame dalam list
# for i in range(len(hasil_co_occurrence_df)):
#   # Iterasi melalui setiap kolom DataFrame
#   for j in range(len(hasil_co_occurrence_df[i].columns)):
#       # Membuat graf hanya untuk nilai di atas treshold
#       edges_above_threshold = hasil_co_occurrence_df[i][hasil_co_occurrence_df[i][hasil_co_occurrence_df[i].columns[j]] > treshold].index.tolist()
#       G = nx.from_pandas_adjacency(hasil_co_occurrence_df[i].loc[edges_above_threshold, edges_above_threshold])

#       # Menambahkan graf ke list hasil
#       graf_results.append(G)


In [ ]:
len(graf_results)

In [ ]:
# menampilkan graf

for i in range(len(graf_results[:3])):
  pos = nx.spring_layout(graf_results[i])  ## Menentukan posisi/koordinat simpul
  labels = nx.get_edge_attributes(graf_results[i], 'weight') ##mendapatkan atribut berbobot ('weight') dari setiap sisi dalam graf.

  # Menentukan ukuran canvas
  plt.figure(figsize=(50, 30))

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(graf_results[i], pos, with_labels=True, node_size=2000, node_color='skyblue')
  nx.draw_networkx_edge_labels(graf_results[i], pos, edge_labels=labels, font_color='red')
  print(f"====== Dokumen ke - {i} ===========")
  plt.show()


## Hitung Centrality

#### Pagerank
> Perhitungan manual  
> Algoritma (Rumus)  
> Perbandingan hasil manual dan library  

In [ ]:
centrality_result = []

for i in range(len(graf_results)):

  centrality = nx.pagerank(graf_results[i]) #menjadikan graf setiap index list diclosness

  centrality_result.append(centrality) #menyimpan hasil centrality


In [ ]:
# menampilkan centrality
for i in range(len(centrality_result[:5])):
  print(f"================== Dokumen ke - {i} ==================")
  print(f"Text rank score : {centrality_result[i]}")

#### Ranking kata kunci

In [ ]:
sorted_closeness_all = []
for i in range(len(centrality_result)):
  sorted_closeness = dict(sorted(centrality_result[i].items(), key=lambda item: item[1], reverse=True))
  sorted_closeness_all.append(sorted_closeness)

sorted_closeness_all[2]

In [ ]:
#kalimat tertinggi

kalimat_tertinggi_all = []

for i in range(len(sorted_closeness_all)):
    kalimat_tertinggi = []
    for j in range(5):
        max_key = None
        max_value = None

        for key, value in sorted_closeness_all[i].items():
            if key not in kalimat_tertinggi and (max_value is None or value > max_value):
                max_key = key
                max_value = value

        if max_key is not None:
            kalimat_tertinggi.append(max_key)

    kalimat_tertinggi_all.append(kalimat_tertinggi)

In [ ]:
len(kalimat_tertinggi_all[0])

In [ ]:
for i in range(len(kalimat_tertinggi_all[:10])):
  print(f"====== Dokumen ke-{i} =====")
  print(f'Kata kunci : {kalimat_tertinggi_all[i]}')
  print('\n')

In [ ]:
# hc = nx.pagerank(graf_results[0])
# hc = dict(sorted(hc.items(), key=lambda item: item[1], reverse=True))

In [ ]:
# hc = nx.pagerank(graf_results[0])
# hc

In [ ]:
# wordss = []
# for i in data['tokenizing'][0]:
#   wordss.append(i.split())

In [ ]:
# # Inisialisasi kamus untuk menyimpan skor
# skor_wordss = {}

# # Loop untuk setiap kata di wordss[0]
# for kata in wordss[2]:
#     # print('word : ',kata)
#     # Jika kata ada di var1, ambil skornya; jika tidak, set skor menjadi 0
#     if kata in hc:
#         skor_wordss[kata] = hc[kata]
#         print('hc:',hc[kata])
#     else:
#         skor_wordss[kata] = 0

# # print(skor_wordss)

In [ ]:
centrality_result[0]

In [ ]:
data['tokenizing'][0]

## Meranking kata sesuai kalimat artikel

In [ ]:
result_skor = []

for i_c in range(len(centrality_result)): #perulangan setiap hasil centrality
  current_skor = [] #untuk menyimpan skor sementara

  for sentences in data['tokenizing'][i_c]: #perulangan kalimat pada data dalam kolom tokenize
    skor_kata = {} #untuk menyimpan skor kata sementara

    words = sentences.split() #menjadikan kalimat berupa kata
    pin_centrality = centrality_result[i_c] #assign centrality indeks sekarang 1,2,3,4...
    panjang_words = len(words) #panjang kata

    for hasil_word in words:

      if hasil_word in pin_centrality:
        skor_kata[hasil_word] = pin_centrality[hasil_word]

      else:
        skor_kata[hasil_word] = 0
    current_skor.append(skor_kata)
  result_skor.append(current_skor)


In [ ]:
result_skor[0]

In [ ]:
result_skor[0]

In [ ]:
result_bobot_kalimat = []

for i in range(len(result_skor)): #perulangan result skor
  current_bobot = {} #untuk menyimpan hasil kalimat dan bobotnya

  for j in range(len(result_skor[i])):
    bobot_kalimat = sum(result_skor[i][j].values())
    current_bobot[data['tokenizing'][i][j]] = bobot_kalimat

  result_bobot_kalimat.append(current_bobot)


In [ ]:
# result_bobot_kalimat

In [ ]:
final_kalimat = []

for i in range(len(result_bobot_kalimat)):
  # Mengurutkan dictionary berdasarkan nilai (dari besar ke kecil)
  sorted_data = dict(sorted(result_bobot_kalimat[i].items(), key=lambda item: item[1], reverse=True))

  # Mengambil 5 nilai teratas beserta key-nya
  top_5_keys = {k: sorted_data[k] for k in list(sorted_data)[:5]}

  # Menggabungkan key menjadi satu string
  result_string = ' '.join(top_5_keys)

  final_kalimat.append(result_string)



In [ ]:
# tampilkan
for i in range(len(final_kalimat)):
  print(f'===== Dokumen {i} ======')
  print(final_kalimat[i])
  print('\n')

### Save

In [ ]:
datas = []
for i in range(len(final_kalimat)):
  datas.append({
      'Summary' : final_kalimat[i],
      'Label' : data['Label'][i]
  })

In [ ]:
import csv
# Menyimpan data dalam bentuk CSV
csv_filename = 'data_textrank_antaranews.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Summary', 'Label']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    for entry in datas:
        csv_writer.writerow(entry)

print(f"Data telah disimpan dalam file {csv_filename}")

In [ ]:
textrank = pd.read_csv('data_textrank_antaranews.csv')
textrank

## Ex sentence tfidf, coocurence, graf

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.shortest_paths import weighted

In [ ]:
x = ['saya pergi ke pasar','saya makan nasi','saya naik sepeda']
# x = [['saya pergi ke pasar'],['saya makan nasi'],['saya naik sepeda']]

In [ ]:
x[0]

In [ ]:
# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Melakukan transformasi TF pada kolom 'final_abstrak'
ex_tf_matrix = vectorizer.fit_transform(x)
terms = vectorizer.get_feature_names_out()
# Membuat DataFrame dari hasil TF
ex_tf_df = pd.DataFrame(ex_tf_matrix.toarray(), columns=terms)

# Cetak DataFrame TF



In [ ]:
ex_tf_df

In [ ]:
# Membuat Co-occurrence Matrix
ex_co_occurrence_matrix = np.dot(ex_tf_df.T, ex_tf_df)

# Konversi ke dense matrix
# ex_co_occurrence_matrix = ex_co_occurrence_matrix.toarray()

# Mengganti diagonal dengan nol (karena kita tidak ingin memperhitungkan kata dengan dirinya sendiri)
np.fill_diagonal(ex_co_occurrence_matrix, 0)

# # Membuat DataFrame Co-occurrence
ex_co_occurrence_df = pd.DataFrame(ex_co_occurrence_matrix, index=terms, columns=terms)

# # Tampilkan DataFrame Co-occurrence
# print("Co-occurrence Matrix:")
ex_co_occurrence_df

In [ ]:
ex_co_occurrence_matrix

In [ ]:
terms

In [ ]:
u = hasil_df_tf[0]
v = u.T.dot(u)
v

In [ ]:
v.values[(np.r_[:len(v)], ) * 2] = 0
v

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Membuat graf dari matriks co-occurrence
graf_result_coba = nx.from_pandas_adjacency(ex_co_occurrence_df)

# # Menambahkan edge dan weight dari matriks co-occurrence
# for i in range(len(terms)):
#     for j in range(i + 1, len(terms)):
#         if co_occurrence_matrix[i, j] > 0:
#             graf_result_coba.add_edge(terms[i], terms[j], weight=co_occurrence_matrix[i, j])

# # Menyiapkan posisi node untuk tampilan yang baik
# pos = nx.spring_layout(graf_result_coba)

# Menggambar graf
nx.draw(graf_result_coba, pos, with_labels=True, font_weight='bold', node_size=700, node_color='skyblue', font_size=8)

# Menambahkan label weight pada setiap edge
edge_labels = nx.get_edge_attributes(graf_result_coba, 'weight')
nx.draw_networkx_edge_labels(graf_result_coba, pos, edge_labels=edge_labels)

# Menampilkan graf
plt.show()


In [ ]:
# Cuma cek Hitung closeness centrality pada index 0
closeness_centrality_coba = nx.pagerank(graf_result_coba)
closeness_centrality_coba

In [ ]:
rankingcoba = dict(sorted(closeness_centrality_coba.items(), key=lambda item: item[1], reverse=True))
rankingcoba

In [ ]:
# Cuma cek Hitung closeness centrality pada index 0
closeness_centrality_coba2 = nx.closeness_centrality(graf_result_coba)
closeness_centrality_coba2

In [ ]:
rankingcoba2 = dict(sorted(closeness_centrality_coba.items(), key=lambda item: item[1], reverse=True))
rankingcoba2

# Modeling

## EDA (Explorasi Data Analysis)  
> proses analisis awal yang dilakukan pada dataset untuk memahami karakteristik, pola, dan struktur data sebelum melakukan analisis lebih lanjut atau membangun model.

### 1.Download dataset

In [ ]:
import pandas as pd
import numpy as np
import gdown

In [ ]:
## download data
nama_data = 'datatextrank.csv'
gdown.download(f'https://drive.google.com/uc?id=1eSvQ-SREetN-UkUblym3tYshrOjSvCAM', nama_data, quiet=False)

### 2.Baca dataset

In [ ]:
datatextrank = pd.read_csv("datatextrank.csv")
datatextrank

#### Banyak dataset

In [ ]:
count_ekonomi = 0
count_olahraga= 0
count_politik = 0

for value in datatextrank['Label']:
  if value == 'ekonomi':
    count_ekonomi+=1
  elif value == 'olahraga':
    count_olahraga+=1
  else:
    count_politik += 1

print(
f'''
Banyak data ekonomi = {count_ekonomi} data
Banyak data olahraga= {count_olahraga} data
Banyak data politik = {count_politik} data

Total Data          = {len(datatextrank)} data
'''
)

## Preprocessing

In [ ]:
### 1.Missing value
datatextrank.isna().sum()

### Cleaning

In [ ]:
# recruitments punctuation
import string

# membuat kolom baru dengan nama new_abstrak untuk data baru yang dipunctuation
datatextrank['summary_punct'] = datatextrank['Summary'].str.replace('[{}]'.format(string.punctuation), '').str.lower()

# Menghilangkan angka dari kolom 'new_abstrak'
datatextrank['summary_punct'] = datatextrank['summary_punct'].str.replace('\d+', '', regex=True)

In [ ]:
datatextrank

#### **Stopword**

> Stopwords digunakan untuk menghilangkan kata umum yang sering muncul dalam teks seperti: di, dan, atau, dari, ke, saya.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

# Download kamus stop words
nltk.download('stopwords')

In [ ]:
# Inisialisasi kamus stop words dari NLTK
stop_words = set(stopwords.words('indonesian'))  # Inisialisasi kamus stop words

# Menghapus stop words dari kolom 'Abstrak'
for stop_word in stop_words:
  datatextrank['summary_stopwords'] = datatextrank['summary_punct'].str.replace(rf'\b{stop_word}\b', '', regex=True) #rf untuk formating string

In [ ]:
datatextrank

#### TFIDF  
> metode yang digunakan dalam pemrosesan bahasa alami (Natural Language Processing - NLP) dan pengambilan informasi (Information Retrieval) untuk mengukur seberapa penting suatu kata dalam suatu dokumen atau kumpulan dokumen. Tujuannya adalah untuk memberikan bobot yang lebih tinggi kepada kata-kata yang penting dalam representasi teks.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Inisialisasi TfidfVectorizer
summ_tfidf_vectorizer = TfidfVectorizer()

# Melakukan transformasi TF-IDF pada kolom 'final_abstrak'
summ_tfidf_matrix = summ_tfidf_vectorizer.fit_transform(datatextrank['summary_stopwords'])

# Membuat DataFrame dari hasil TF-IDF
summ_tfidf_df = pd.DataFrame(summ_tfidf_matrix.toarray(),columns=summ_tfidf_vectorizer.get_feature_names_out())

summ_tfidf_df = pd.concat([datatextrank['Summary'],summ_tfidf_df,datatextrank['Label']],axis=1)
# Cetak DataFrame TF-IDF
summ_tfidf_df

In [ ]:
# pd.DataFrame(summ_tfidf_df['Summary'])

In [ ]:
# summ_tfidf_df.iloc[:,1:-1]

### Model

#### Split dataset

In [ ]:
# import library
from sklearn.model_selection import train_test_split

In [ ]:
# Bagi data menjadi pelatihan (80%) dan pengujian (20%)
X_tfidf = summ_tfidf_matrix
# X_tfidf
y_tfidf = summ_tfidf_df["Label"]
# y_tfidf
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf, y_tfidf, test_size=0.2, random_state=42)


In [ ]:
len(X_tfidf_train.toarray())
len(y_tfidf_train)

In [ ]:
# print(len(y_tfidf_test),len(X_tfidf_test))

In [ ]:
y_tfidf_train.info()

#### Naive Bayes

In [ ]:
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Contoh penggunaan
naive_bayes = GaussianNB()

# Konversi matriks sejumput ke matriks padat untuk data pelatihan
X_dense_train = X_tfidf_train.toarray()
naive_bayes.fit(X_dense_train, y_tfidf_train)

# Konversi matriks sejumput ke matriks padat untuk data pengujian
X_dense_test = X_tfidf_test.toarray()
y_pred = naive_bayes.predict(X_dense_test)

accuracy_naiveBayes = accuracy_score(y_tfidf_test, y_pred)
print(f'Akurasi model Naive Bayes: {accuracy_naiveBayes}')

#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()

X_dense_train = X_tfidf_train.toarray()

# Latih model pada kolom topik tertentu
knn.fit(X_dense_train, y_tfidf_train)

# Lakukan prediksi
y_pred = knn.predict(X_tfidf_test)

accuracy_knn = accuracy_score(y_tfidf_test, y_pred)
print(f'Akurasi model KNN: {accuracy_knn}')

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Inisialisasi model Naive Bayes
rf = RandomForestClassifier()

X_dense_train = X_tfidf_train.toarray()

# Latih model pada kolom topik tertentu
rf.fit(X_dense_train, y_tfidf_train)

# Lakukan prediksi
y_pred = rf.predict(X_tfidf_test)

accuracy_rf = accuracy_score(y_tfidf_test, y_pred)
print(f'Akurasi model Random Forest: {accuracy_rf}')

### Evaluasi

In [ ]:
import matplotlib.pyplot as plt

models = ['Naive Bayes', 'KNN', 'Random Forest']
accuracies = [accuracy_naiveBayes, accuracy_knn, accuracy_rf]

plt.bar(models, accuracies, color=[ 'green', 'blue', 'red'])
plt.ylabel('Akurasi Model')
plt.title('Akurasi Model Machine Learning')
plt.ylim(0, 1)  # Sesuaikan dengan rentang akurasi (0-1)
plt.grid()
plt.show()

### Implementasi

In [ ]:
# teks = ['Jakarta - Makna motif adalah alasan atau penyebab seseorang melakukan sesuatu.',
#         'Ketika kita melakukan suatu tindakan pasti ada sesuatu hal yang mendorongnya.',
#         'Contohnya kita makan karena kita lapar.',
#         'Hal tersebut juga terjadi pada kegiatan ekonomi.',
#         'Semua kegiatan ekonomi tidak dapat terjadi begitu saja, namun terdapat suatu alasan atau penyebab yang mendorongnya.',
#         'Faktor pendorong inilah yang disebut motif ekonomi.',
#         'Baca artikel detikfinance, "Pengertian Motif Ekonomi serta Tujuan, Sifat, Macam, dan Contohnya" selengkapnya https://finance.detik.com/berita-ekonomi-bisnis/d-7048370/pengertian-motif-ekonomi-serta-tujuan-sifat-macam-dan-contohnya.',
#         'Download Apps Detikcom Sekarang https://apps.detik.com/detik/']

# teks_lower = [kalimat.lower() for kalimat in teks]

# print(teks_lower)


In [ ]:
# ========== data inputan ==========
new_data = str(input("masukkan data: "))

# ========== tokenizing kalimat ==========
new_data = sent_tokenize(new_data)
new_data = [kalimat.lower() for kalimat in new_data] #menjadikan lower

# ========== vectorize ==========
inp_hasil_df_tf = []

inp_termFreq_vectorizer = CountVectorizer()  #inisialisasi tfidf
inp_termFreq_matrix = inp_termFreq_vectorizer.fit_transform(new_data) #menjadikan tfidf setiap dokumen

inp_terms = inp_termFreq_vectorizer.get_feature_names_out()
inp_tf_df = pd.DataFrame(inp_termFreq_matrix.toarray(), columns=inp_terms)

inp_hasil_df_tf.append(inp_tf_df)


Jakarta - Makna motif adalah alasan atau penyebab seseorang melakukan sesuatu. Ketika kita melakukan suatu tindakan pasti ada sesuatu hal yang mendorongnya. Contohnya kita makan karena kita lapar. Hal tersebut juga terjadi pada kegiatan ekonomi. Semua kegiatan ekonomi tidak dapat terjadi begitu saja, namun terdapat suatu alasan atau penyebab yang mendorongnya. Faktor pendorong inilah yang disebut motif ekonomi.  Baca artikel detikfinance, "Pengertian Motif Ekonomi serta Tujuan, Sifat, Macam, dan Contohnya" selengkapnya https://finance.detik.com/berita-ekonomi-bisnis/d-7048370/pengertian-motif-ekonomi-serta-tujuan-sifat-macam-dan-contohnya.  Download Apps Detikcom Sekarang https://apps.detik.com/detik/

JAKARTA, Indotimes.co.id – Putri Zulkifli Hasan, anak dari Ketua Umum Partai Amanat Nasional (PAN) Zulkifli Hasan, tak punya itikad baik untuk menyelesaikan sengketa lahan yang menyeret namanya dengan cara mediasi.

Dengan demikian, masalah sengketa tanah itu berlanjut dalam persidangan yang harus mendatangkan pihak-pihak beperkara.

Sidang yang melibatkan Putri Zulkifli Hasan ini kembali digelar di Pengadilan Negeri Jakarta Timur (PN Jaktim), Kamis (9/11).

Para pihak yang hadir, antara lain penggugat I Aziz Anugerah Yudha Prawira yang diwakili kuasa hukumnya dari Firma Hukum DR. Yayan Riyanto, SH, MH.

Adapun tergugat yang hadir, yaitu Tergugat I Lie Andry Setyadarma yang diwakili pengacaranya dan Tergugat III Putri Zulkifli Hasan yang juga diwakili kuasa hukumnya, Andreas Tambunan.

DR. Yayan Riyanto menyampaikan, gugatan yang diajukannya sudah dibacakan majelis hakim untuk kemudian dijawab para pihak tergugat.

In [ ]:
# inp_hasil_df_tf

In [ ]:
# ========== co ocurence =======
inp_hasil_co_occurrence_df = []
inp_hasil_co_occurrence_matrix = []

for i in range(len(inp_hasil_df_tf)):

  # Membuat Co-occurrence Matrix
  inp_co_occurrence_matrix = np.dot(inp_hasil_df_tf[i].T,inp_hasil_df_tf[i])

  # Mengganti diagonal dengan nol (karena kita tidak ingin memperhitungkan kata dengan dirinya sendiri)
  np.fill_diagonal(inp_co_occurrence_matrix, 0)

  # mengambil columns
  inp_kolom = inp_hasil_df_tf[i].columns

  # # Membuat DataFrame Co-occurrence
  inp_co_occurrence_df = pd.DataFrame(inp_co_occurrence_matrix, index=inp_kolom, columns=inp_kolom)

  inp_hasil_co_occurrence_df.append(inp_co_occurrence_df)
  inp_hasil_co_occurrence_matrix.append(inp_co_occurrence_matrix)


In [ ]:
inp_hasil_co_occurrence_df

In [ ]:
# ========== tambah graf dan atur treshold =======
inp_graf_results = []
treshold = 0

for i in range(len(inp_hasil_co_occurrence_df)):

  inp_graf = nx.Graph()  # Instantiate as a Graph object

  inp_co_occurrence = inp_hasil_co_occurrence_df[i] #mengambil hasil coocurence per indeks yang disimpan array
  inp_koloms = inp_co_occurrence.columns #mengambil nama kolom saja dari coocurence array indeks
  inp_matrix_coocurrence = inp_hasil_co_occurrence_matrix[i] #mengambil matrix coocurencenya

  for i_koloms in range(len(inp_koloms)): #perulangan setiap dari panjang kolom dari coocurence
      for j_koloms in range(i_koloms + 1, len(inp_koloms)): #perulangan dari setiap panjang kolom + 1, maksudna kolom indeks ke dua

        inp_bobot = inp_matrix_coocurrence[i_koloms,j_koloms] #mencari bobot dengan setiap kolom

        if inp_bobot > treshold: #cek bobot
          inp_graf.add_edge(inp_koloms[i_koloms], inp_koloms[j_koloms], weight=inp_matrix_coocurrence[i_koloms, j_koloms])

  inp_graf_results.append(inp_graf)


In [ ]:
# ==== centrality ===
inp_centrality_result = []

for i in range(len(inp_graf_results)):
  inp_centrality = nx.pagerank(inp_graf_results[i]) #menjadikan graf setiap index list diclosness

  inp_centrality_result.append(inp_centrality)
  # print(i)

In [ ]:
len(inp_centrality_result)

In [ ]:
# ========= ranking ==========
inp_result_skor = []

for sentences in new_data:

  words = sentences.split()
  skor_kata = {}

  for cek in words:
    if cek in inp_centrality_result[0]:
      skor_kata[cek] = inp_centrality_result[0][cek]
  inp_result_skor.append(skor_kata)


In [ ]:
len(inp_result_skor)

In [ ]:
# inp_result_skor

In [ ]:
inp_result_bobot_kalimat = []

for i in range(len(inp_result_skor)):
    current_bobot = {}

    bobot_kalimat = sum(inp_result_skor[i].values())
    current_bobot[new_data[i]] = bobot_kalimat

    inp_result_bobot_kalimat.append(current_bobot)


In [ ]:
inp_result_bobot_kalimat

In [ ]:
current_inp = []
inp_sorted_data = sorted(inp_result_bobot_kalimat, key=lambda x: list(x.values())[0], reverse=True)

# ambil 5 data teratas
inp_sorted_data = inp_sorted_data[:5]

final_inp = []
for hasil in inp_sorted_data:
  for key,values in hasil.items():
    final_inp.append(key)


In [ ]:
# final_inp

In [ ]:
final_inp = "".join(final_inp)


In [ ]:
# === tfidf ===

# Membuat final_inp menjadi list jika belum
if not isinstance(final_inp, list):
    final_inp = [final_inp]

# Melakukan transformasi TF-IDF pada final_inp
rank_inp = summ_tfidf_vectorizer.transform(final_inp)
# rank_inp.toarray()


In [ ]:

z = str(input('masukkan kata : '))

# === tfidf ===

# Membuat z menjadi list jika belum
if not isinstance(z, list):
    z = [z]

# Melakukan transformasi TF-IDF pada z
rank_z = summ_tfidf_vectorizer.transform(z)
# rank_inp.toarray()


predict_inp = knn.predict(rank_inp.toarray())
predict_inp

In [ ]:
# test

predict_inp = knn.predict(rank_inp.toarray())
predict_inp